# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_v_py = pd.read_csv('weather.csv')
weather_v_py

,city,lat,temp,humidity,clouds,wind,lon
0,tasiilaq,65.6145,37.40,81.0,90.0,11.50,-37.6368
1,bajo baudo,19.7297,75.07,78.0,90.0,6.91,-155.0900
2,hilo,18.0858,68.00,77.0,0.0,14.97,-15.9785
3,nouakchott,29.0361,59.92,95.0,100.0,1.34,114.5636
4,yining,-53.1500,48.09,87.0,90.0,13.80,-70.9167
...,...,...,...,...,...,...,...
605,santa cruz,NaN,NaN,NaN,NaN,NaN,NaN
606,alto araguaia,NaN,NaN,NaN,NaN,NaN,NaN
607,ola,NaN,NaN,NaN,NaN,NaN,NaN
608,west bay,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
weather_v_py = weather_v_py.dropna()
city = weather_v_py["city"]
locations = weather_v_py[["lat", "lon"]].astype(float)
humidity = weather_v_py["humidity"]

fig = gmaps.figure()
m = gmaps.Map()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
new_df = weather_v_py.loc[(weather_v_py['temp'] < 80) & (weather_v_py['temp'] > 70) & (weather_v_py['wind'] < 10) & (weather_v_py['clouds'] == 0)]
new_df

,city,lat,temp,humidity,clouds,wind,lon
98,sistranda,-12.7336,78.80,89.0,0.0,3.44,45.2042
160,castro,-20.7377,75.65,49.0,0.0,7.87,116.8463
308,karanja,-15.1961,77.59,65.0,0.0,8.34,12.1522
337,namibe,27.2167,78.46,12.0,0.0,4.09,77.4833
364,lhokseumawe,20.5169,79.54,18.0,0.0,3.49,-13.0499
444,cardston,26.7292,75.20,60.0,0.0,6.20,33.9365
512,hay river,38.4362,75.15,28.0,0.0,6.40,57.4316
529,vangaindrano,22.9167,70.92,53.0,0.0,2.82,96.5000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
